In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import matplotlib.pyplot as plt
import torch.nn.utils.rnn as rnn_utils

In [ ]:
def collate_fn(batch):
    features, gaze_direction = zip(*batch)

    # Pad sequences to the length of the longest sequence in the batch
    padded_features = rnn_utils.pad_sequence(features, batch_first=True, padding_value=0)
    padded_gaze_direction = rnn_utils.pad_sequence(gaze_direction, batch_first=True, padding_value=0)

    return padded_features, padded_gaze_direction

def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data), scaler

In [ ]:
class GazeDirectionDataset(Dataset):
    def __init__(self, folder_path, sequence_length=5):
        self.sequence_length = sequence_length
        self.data = []
        self.feature = []

        # Load and aggregate data from all CSV files in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                data = pd.read_csv(file_path)
                features = data[['HeadX', 'HeadY', 'HeadZ','REyeRX', 'REyeRY', 'REyeRZ']].values
                gaze_data = data[['REyeRX', 'REyeRY', 'REyeRZ']].values
                self.data.append(gaze_data)
                self.feature.append(features)

        # Concatenate and normalize the data
        all_data = np.concatenate(self.data, axis=0)
        all_data1 = np.concatenate(self.feature, axis=0)

        self.normalized_features, self.scaler_features = normalize_data(all_data1)

        self.normalized_gaze_data, self.scaler = normalize_data(all_data)

    def __len__(self):
        return len(self.normalized_gaze_data) - self.sequence_length - 1

    def __getitem__(self, idx):
        input_seq = self.normalized_features[idx:idx + self.sequence_length]
        target_seq = self.normalized_gaze_data[idx + 1:idx + self.sequence_length + 1]

        input_seq = torch.tensor(input_seq, dtype=torch.float32)
        target_seq = torch.tensor(target_seq, dtype=torch.float32)

        return input_seq, target_seq

In [ ]:
# class GazeDirectionDataset(Dataset):
#     def __init__(self, data_dir):
#         self.data_dir = data_dir
#         self.file_list = [file for file in os.listdir(data_dir) if file.endswith('.csv')]

#     def __len__(self):
#         return len(self.file_list)

#     def __getitem__(self, idx):
#         file_name = self.file_list[idx]
#         file_path = os.path.join(self.data_dir, file_name)

#         # Load the data from the text file with the correct delimiter
#         data = pd.read_csv(file_path, delimiter=',')  # Assuming comma-separated values

#         # Extract features and labels (assuming 'GazeDirection' is the column to predict)
#         features = data[['HeadRX', 'HeadRY', 'HeadRZ']].values[::100]   # Exclude 'Timer' and gaze direction columns
#         # features = data[['HeadX', 'HeadY', 'HeadZ','HeadRX', 'HeadRY', 'HeadRZ']].values  # Exclude 'Timer' and gaze direction columns

#         gaze_direction = data[['REyeRX', 'REyeRY', 'REyeRZ']].values[::100]

#         # Convert to PyTorch tensors
#         features = torch.tensor(features, dtype=torch.float32)
#         gaze_direction = torch.tensor(gaze_direction, dtype=torch.float32)

#         return features, gaze_direction

In [ ]:
# class Encoder(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(Encoder, self).__init__()
#         self.latent_dim = latent_dim
#         self.lstm1 = nn.LSTM(input_dim, self.latent_dim, batch_first=True)
#         self.lstm2 = nn.LSTM(self.latent_dim, self.latent_dim, batch_first=True)

#     def forward(self, src):
#         print("encoder",src.shape)
#         outputs, (hidden, cell) = self.lstm1(src)
#         outputs, (hidden, cell) = self.lstm2(outputs)
#         return hidden, cell

# class Decoder(nn.Module):
#     def __init__(self, output_dim, latent_dim):
#         super(Decoder, self).__init__()
#         self.latent_dim = latent_dim
#         self.lstm1 = nn.LSTM(output_dim, self.latent_dim, batch_first=True)
#         self.lstm2 = nn.LSTM(self.latent_dim, self.latent_dim, batch_first=True)
#         self.fc_out = nn.Linear(self.latent_dim, output_dim)

#     def forward(self, trg, hidden, cell):
#         print("decoder",trg.shape)

#         # trg = trg.unsqueeze(1)
#         output, (hidden, cell) = self.lstm1(trg, (hidden, cell))
#         output, (hidden, cell) = self.lstm2(output, (hidden, cell))
#         prediction = self.fc_out(output)
#         return prediction, hidden, cell

# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder, device):
#         super(Seq2Seq, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#         self.device = device

#     def forward(self, src, trg, teacher_forcing_ratio = 0.5):
#         print("main", src.shape, trg.shape)

#         batch_size = trg.shape[0]
#         trg_len = trg.shape[1]
#         trg_vocab_size = self.decoder.fc_out.out_features

#         outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
#         hidden, cell = self.encoder(src)
#         # hidden = hidden.unsqueeze(1)
#         # cell = cell.unsqueeze(1)

#         # input = trg[:, 0, :]
#         input = trg[:, 0, :].unsqueeze(1).float()
#         for t in range(1, trg_len):
#             output, hidden, cell = self.decoder(input, hidden, cell)
#             outputs[:,t,:] = output
#             top1 = output.argmax(1)
#             input = trg[:, t,:] if random.random() < teacher_forcing_ratio else top1
#         # for t in range(1, trg_len):
#         #   output, hidden, cell = self.decoder(input, hidden, cell)
#         #   print(f"Decoder output shape at timestep {t}: {output.shape}")

#         #   outputs[:, t, :] = output.squeeze(1)
#         #   if output.shape[2] > 1:  # If more than one feature
#         #       top1 = output.argmax(2)  # Reduces to [batch_size, 1]
#         #       input = trg[:, t, :].unsqueeze(1) if random.random() < teacher_forcing_ratio else top1.unsqueeze(1)  # Reshape to [batch_size, 1, features]
#         #   else:
#         #       # If only one feature, use output directly as next input
#         #       input = trg[:, t, :].unsqueeze(1) if random.random() < teacher_forcing_ratio else output

#         return outputs


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.0, batch_first=True):
        super(Seq2Seq, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Encoder
        self.encoder_lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)

        # Decoder
        self.decoder_lstm = nn.LSTM(output_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)

        # Dense layer for prediction
        self.decoder_dense = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_input, decoder_input):
        # Encoder
        _, (hidden, cell) = self.encoder_lstm(encoder_input)

        # Decoder
        decoder_output, _ = self.decoder_lstm(decoder_input, (hidden, cell))

        # Final prediction
        output = self.decoder_dense(decoder_output)

        return output

    def init_hidden(self, batch_size):
        # This is used to initialize the hidden and cell states if needed
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell

In [ ]:
class Seq2Seq_nlpmixing(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq_nlpmixing, self).__init__()
        self.encoder1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.encoder2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.decoder_lstm1 = nn.LSTM(output_size, hidden_size, batch_first=True)
        self.decoder_lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.decoder_dense = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_input, decoder_input, encoder_lengths=None):
        encoder1_outputs, _ = self.encoder1(encoder_input)
        encoder2_outputs, _ = self.encoder2(encoder1_outputs)

        decoder1_outputs, _ = self.decoder_lstm1(decoder_input)
        decoder2_outputs, _ = self.decoder_lstm2(decoder1_outputs)

        output = self.decoder_dense(decoder2_outputs)

        return output

In [ ]:
# Split the dataset into training and validation sets
data_directory = os.path.expanduser("/content/drive/MyDrive/colab/ECE6123_Final_Project/processed_by_activity/presenting")
# custom_dataset = GazeDirectionDataset(data_directory)
custom_dataset = GazeDirectionDataset(data_directory)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset, val_dataset = train_test_split(custom_dataset, test_size=0.3, random_state=42, shuffle=False)
val_dataset, test_dataset = train_test_split(val_dataset, test_size=0.5, random_state=42)
input_size = 6  # Update based on your input features
hidden_size = 128
output_size = 3 # Update based on your output features
model = Seq2Seq(input_size, hidden_size, output_size).to(device)
learning_rate = 0.001
batch_size = 32
# Instantiate the model
# encoder = Encoder(INPUT_DIM, LATENT_DIM)
# decoder = Decoder(OUTPUT_DIM, LATENT_DIM)
# model = Seq2Seq(encoder, decoder, device).to(device)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# check back if scheduler is needed
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6, verbose=True)

# Early stopping parameters
early_stopping_patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# cuda
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 30
train_losses = []
val_losses = []
for src, trg in train_dataloader:
  print(src.shape, trg.shape)
  break
print(custom_dataset,custom_dataset,custom_dataset)
# print(train_dataloader[0])

In [ ]:
# Split the dataset into training and validation sets
data_directory = os.path.expanduser("/content/drive/MyDrive/colab/ECE6123_Final_Project/processed_by_activity/sweep")
# custom_dataset = GazeDirectionDataset(data_directory)
custom_dataset = GazeDirectionDataset(data_directory)

train_dataset, val_dataset = train_test_split(custom_dataset, test_size=0.3, random_state=42, shuffle=True)
val_dataset, test_dataset = train_test_split(val_dataset, test_size=0.5, random_state=42)
# Data loaders
# Instantiate the model
input_size = 6 # Update based on your input features
hidden_size = 64
output_size = 3 # Update based on your output features
seq2seq_model = Seq2Seq_nlpmixing(input_size, hidden_size, output_size)
learning_rate = 0.0001
batch_size = 32


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(seq2seq_model.parameters(), lr=learning_rate)
# check back if scheduler is needed
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6, verbose=True)

# Early stopping parameters
early_stopping_patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# cuda
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seq2seq_model.to(device)

# Training loop
num_epochs = 30
train_losses = []
val_losses = []


for epoch in range(num_epochs):
    total_train_loss = 0.0
    model.train()

    for i, batch in enumerate(train_dataloader):
        features, gaze_direction = batch
        features, gaze_direction = features.to(device).float(), gaze_direction.to(device).float()
        # Assuming decoder_input is the same as target for simplicity (modify as needed)
        optimizer.zero_grad()
        output = model(features, gaze_direction)
        # output = seq2seq_model(gaze_direction)

        loss = criterion(output, gaze_direction)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        # print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_dataloader)}], Training Loss: {loss.item()}')
    # Validation
    model.eval()

    with torch.no_grad():
        total_val_loss = 0.0
        for batch in val_dataloader:
            features, gaze_direction = batch
            features, gaze_direction = features.to(device), gaze_direction.to(device)
            decoder_input = gaze_direction
            # decoder_input = torch.zeros_like(gaze_direction)  # Initialize decoder input, replace with your logic if needed
            val_output = model(features, decoder_input)
            total_val_loss += criterion(val_output, gaze_direction).item()

    average_train_loss = total_train_loss / len(train_dataloader)
    average_val_loss = total_val_loss / len(val_dataloader)
    train_losses.append(average_train_loss)
    val_losses.append(average_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {average_train_loss}, Average Validation Loss: {average_val_loss}')
    # scheduler.step()
    scheduler.step(average_val_loss)

    # Early stopping check
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Save the model if needed
torch.save(seq2seq_model.state_dict(), 'seq2seq_mlp.pth')

In [ ]:

plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('seq2seq + MLP mixing: Cleaning')
plt.legend()
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

## nlp mixing

class GazeDirectionDataset(Dataset):
    def __init__(self, folder_path, sequence_length=5, num_users=None):
        self.sequence_length = sequence_length
        self.num_users = num_users  # number of users to include in the other_users_data
        self.data = []
        self.features = []
        self.other_users_data = []

        # Load and aggregate data from all CSV files in the folder
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        for filename in csv_files:
            file_path = os.path.join(folder_path, filename)
            data = pd.read_csv(file_path)
            features = data[['HeadX', 'HeadY', 'HeadZ', 'REyeRX', 'REyeRY', 'REyeRZ']].values
            gaze_data = data[['REyeRX', 'REyeRY', 'REyeRZ']].values
            self.data.append(gaze_data)
            self.features.append(features)

        # Here we assume that the first user is the target user
        # and the rest are other users to be used for mixing
        self.normalized_features, self.scaler_features = normalize_data(np.concatenate(self.features, axis=0))
        self.normalized_gaze_data, self.scaler = normalize_data(np.concatenate(self.data, axis=0))

        num_samples = self.normalized_gaze_data.shape[0] // len(csv_files)
        num_features = self.normalized_gaze_data.shape[1]

        self.normalized_gaze_data = self.normalized_gaze_data.reshape(len(csv_files), num_samples, num_features)

        # If num_users is provided, select a subset of users for other_users_data
        if num_users is not None and num_users <= len(csv_files):

            # Select a subset of users for other_users_data if num_users is provided
            self.other_users_data = self.normalized_gaze_data[:, :num_users-1, :]
        else:
            # Use data from all other users
            self.other_users_data = self.normalized_gaze_data[:, 1:, :]

    def __len__(self):
        return len(self.normalized_gaze_data) - self.sequence_length - 1

    def __getitem__(self, idx):
        input_seq = self.normalized_features[idx:idx + self.sequence_length]
        target_seq = self.normalized_gaze_data[idx + 1:idx + self.sequence_length + 1]
        # Here we need to provide the other users' data as well
        other_users_seq = self.other_users_data[idx + 1:idx + self.sequence_length + 1]

        input_seq = torch.tensor(input_seq, dtype=torch.float32)
        target_seq = torch.tensor(target_seq, dtype=torch.float32)
        other_users_seq = torch.tensor(other_users_seq, dtype=torch.float32)

        return input_seq, target_seq, other_users_seq


In [ ]:
import torch
import torch.nn as nn

class Seq2SeqWithMLPMixing(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_users, num_layers=2, dropout=0.0, batch_first=True):
        super(Seq2SeqWithMLPMixing, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Encoder
        self.encoder_lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)

        # Decoder
        self.decoder_lstm = nn.LSTM(output_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)

        # Dense layer for prediction
        self.decoder_dense = nn.Linear(hidden_size, output_size)

        # MLP for mixing other users' data
        self.mlp_mixing = nn.Sequential(
            nn.Linear(hidden_size + (num_users - 1) * output_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, encoder_input, decoder_input, other_users_data=None):
        # Encoder
        _, (hidden, cell) = self.encoder_lstm(encoder_input)

        # Decoder
        decoder_output, _ = self.decoder_lstm(decoder_input, (hidden, cell))

        # If other users' data is provided, mix it using MLP
        if other_users_data is not None:
            # Flatten the other users' data to match the batch and feature dimensions
            other_users_data_flat = other_users_data.view(other_users_data.size(0), -1)
            # Concatenate the decoder output with the other users' data
            mixed_input = torch.cat((decoder_output, other_users_data_flat), dim=2)
            # Pass through the MLP mixing layer
            decoder_output = self.mlp_mixing(mixed_input)

        # Final prediction
        output = self.decoder_dense(decoder_output)

        return output



In [ ]:
class GazeDirectionDataset(Dataset):
    def __init__(self, folder_path, sequence_length=5):
        self.sequence_length = sequence_length
        data_list = []
        feature_list = []

        # Load and aggregate data from all CSV files in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                data = pd.read_csv(file_path)
                features = data[['HeadX', 'HeadY', 'HeadZ', 'REyeRX', 'REyeRY', 'REyeRZ']].values
                gaze_data = data[['REyeRX', 'REyeRY', 'REyeRZ']].values
                data_list.append(gaze_data)
                feature_list.append(features)

        # Normalize the data using the external function
        self.normalized_features, self.scaler_features = normalize_data(np.concatenate(feature_list, axis=0))
        self.normalized_gaze_data, self.scaler = normalize_data(np.concatenate(data_list, axis=0))

    def __len__(self):
        return len(self.normalized_gaze_data) - self.sequence_length - 1

    def __getitem__(self, idx):
        # Finding the right sequence in the list of sequences
        for data_seq, feature_seq in zip(self.normalized_gaze_data, self.normalized_features):
            if idx < len(data_seq) - self.sequence_length:
                input_seq = feature_seq[idx:idx + self.sequence_length]
                target_seq = data_seq[idx + 1:idx + self.sequence_length + 1]
                break
            idx -= len(data_seq) - self.sequence_length

        input_seq = torch.tensor(input_seq, dtype=torch.float32)
        target_seq = torch.tensor(target_seq, dtype=torch.float32)

        return input_seq, target_seq

In [ ]:
# Split the dataset into training and validation sets
data_directory = os.path.expanduser("/content/drive/MyDrive/colab/ECE6123_Final_Project/processed_by_activity/sweep")
# custom_dataset = GazeDirectionDataset(data_directory)
custom_dataset = GazeDirectionDataset(data_directory,5)

train_dataset, val_dataset = train_test_split(custom_dataset, test_size=0.3, random_state=42, shuffle=True)
val_dataset, test_dataset = train_test_split(val_dataset, test_size=0.5, random_state=42)
# Data loaders
# Instantiate the model
input_size = 6 # Update based on your input features
hidden_size = 64
output_size = 3 # Update based on your output features
seq2seq_model = Seq2Seq_nlpmixing(input_size, hidden_size, output_size)
learning_rate = 0.0001
batch_size = 32


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(seq2seq_model.parameters(), lr=learning_rate)
# check back if scheduler is needed
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6, verbose=True)

# Early stopping parameters
early_stopping_patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# cuda
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seq2seq_model.to(device)

# Training loop
num_epochs = 30
train_losses = []
val_losses = []


for epoch in range(num_epochs):
    total_train_loss = 0.0
    model.train()

    for i, batch in enumerate(train_dataloader):
        features, gaze_direction = batch
        features, gaze_direction = features.to(device).float(), gaze_direction.to(device).float()
        # Assuming decoder_input is the same as target for simplicity (modify as needed)
        optimizer.zero_grad()
        output = model(features, gaze_direction)
        # output = seq2seq_model(gaze_direction)

        loss = criterion(output, gaze_direction)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        # print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_dataloader)}], Training Loss: {loss.item()}')
    # Validation
    model.eval()

    with torch.no_grad():
        total_val_loss = 0.0
        for batch in val_dataloader:
            features, gaze_direction = batch
            features, gaze_direction = features.to(device), gaze_direction.to(device)
            decoder_input = gaze_direction
            # decoder_input = torch.zeros_like(gaze_direction)  # Initialize decoder input, replace with your logic if needed
            val_output = model(features, decoder_input)
            total_val_loss += criterion(val_output, gaze_direction).item()

    average_train_loss = total_train_loss / len(train_dataloader)
    average_val_loss = total_val_loss / len(val_dataloader)
    train_losses.append(average_train_loss)
    val_losses.append(average_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {average_train_loss}, Average Validation Loss: {average_val_loss}')
    # scheduler.step()
    scheduler.step(average_val_loss)

    # Early stopping check
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Save the model if needed
torch.save(seq2seq_model.state_dict(), 'seq2seq_mlp.pth')

In [ ]:
class GazeDirectionDataset(Dataset):
    def __init__(self, folder_path, sequence_length=5, num_users=None):
        self.sequence_length = sequence_length
        self.gaze_data = []

        # Load and aggregate gaze data from all CSV files in the folder
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        for filename in csv_files:
            file_path = os.path.join(folder_path, filename)
            data = pd.read_csv(file_path)
            gaze_data = data[['REyeRX', 'REyeRY', 'REyeRZ']].values
            self.gaze_data.append(gaze_data)

        # Normalize data
        all_gaze_data = np.concatenate(self.gaze_data, axis=0)
        self.normalized_gaze_data, self.scaler = normalize_data(all_gaze_data)

        # Reshape to have 3 dimensions: (samples, timesteps, features)
        num_samples_per_user = self.normalized_gaze_data.shape[0] // len(csv_files)
        self.normalized_gaze_data = self.normalized_gaze_data.reshape(len(csv_files), num_samples_per_user, -1)

        # Select a subset of users for other_users_data
        if num_users is not None and num_users <= len(csv_files):
            self.other_users_data = self.normalized_gaze_data[1:num_users, :, :]
        else:
            self.other_users_data = self.normalized_gaze_data[1:, :, :]

    def __len__(self):
        return len(self.normalized_gaze_data[0]) - self.sequence_length - 1

    def __getitem__(self, idx):
        if idx + self.sequence_length >= len(self.normalized_gaze_data[0]):
            raise IndexError("Index out of range for sequence generation")

        target_user_seq = self.normalized_gaze_data[0, idx:idx + self.sequence_length]
        target_user_seq = torch.tensor(target_user_seq, dtype=torch.float32)

        other_users_seq = self.other_users_data[:, idx:idx + self.sequence_length]
        other_users_seq = torch.tensor(other_users_seq, dtype=torch.float32)

        return target_user_seq, other_users_seq

In [ ]:
import torch
import torch.nn as nn

class Seq2SeqWithMLPMixing(nn.Module):
    def __init__(self, gaze_data_size, hidden_size, num_users, num_layers=2, dropout=0.0, batch_first=True):
        super(Seq2SeqWithMLPMixing, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Encoder and Decoder use only gaze data
        self.encoder_lstm = nn.LSTM(gaze_data_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)
        self.decoder_lstm = nn.LSTM(gaze_data_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=batch_first)

        # Dense layer for prediction
        self.decoder_dense = nn.Linear(hidden_size, gaze_data_size)

        # MLP for mixing other users' data
        self.mlp_mixing = nn.Sequential(
            nn.Linear(hidden_size + (num_users - 1) * gaze_data_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, gaze_data_size)
        )

    def forward(self, encoder_input, decoder_input, other_users_data=None):
        # Encoder
        _, (hidden, cell) = self.encoder_lstm(encoder_input)

        # Decoder
        decoder_output, _ = self.decoder_lstm(decoder_input, (hidden, cell))

        # If other users' data is provided, mix it using MLP
        if other_users_data is not None:
            other_users_data_flat = other_users_data.view(other_users_data.size(0), -1)
            mixed_input = torch.cat((decoder_output, other_users_data_flat), dim=2)
            decoder_output = self.mlp_mixing(mixed_input)

        # Final prediction
        output = self.decoder_dense(decoder_output)

        return output

# Example usage
gaze_data_size = 3  # Assuming gaze data has 3 features (REyeRX, REyeRY, REyeRZ)
num_users = 5  # Number of users including the target user
latent_dim = 256  # Hidden size for LSTM layers

model = Seq2SeqWithMLPMixing(gaze_data_size, latent_dim, num_users)


In [ ]:
sequence_length = 5
num_users = 5  # Including the target user
batch_size = 32
num_epochs = 10
learning_rate = 0.001
gaze_data_size = 3  # Assuming 3 features for gaze data
latent_dim = 256  # Hidden size for LSTM layers
data_directory = os.path.expanduser("/content/drive/MyDrive/colab/ECE6123_Final_Project/processed_by_activity/sweep")

# Load the dataset
dataset = GazeDirectionDataset(data_directory, sequence_length, num_users)

# Split dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = Seq2SeqWithMLPMixing(gaze_data_size, latent_dim, num_users)
model.train()

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    total_loss = 0
    for target_user_seq, other_users_seq in train_loader:
        # Forward pass
        optimizer.zero_grad()
        output = model(target_user_seq, target_user_seq[:, 1:, :], other_users_seq)
        loss = criterion(output, target_user_seq[:, 1:, :])

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Validation (optional, but recommended)
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for target_user_seq, other_users_seq in val_loader:
            output = model(target_user_seq, target_user_seq[:, 1:, :], other_users_seq)
            loss = criterion(output, target_user_seq[:, 1:, :])
            val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
    print(f'Validation Loss: {avg_val_loss:.4f}')
    model.train()
